# Entorno de trabajo: Crear un dataset de noticias de prensa a partir del motor de búsqueda Sophia

Este notebook muestra una manera de extraer noticias desde el motor de búsqueda de _Sophia_, a partir de una **palabra clave**, un **país** y un **periodo de tiempo**.


_Sophia_ es un proyecto I+i+D desarrollado en la Universidad Austral de Chile desde 2019. El motor de búsqueda se encuentra en fase de mejoramiento continuo. Por cualquier pregunta o problema técnica con este notebook, no dudar en contactarme por Discord (@MatthieuVernier) o por correo mvernier@inf.uach.cl.

## 0. Librerías útiles

- Elasticsearch es el motor de búsqueda utilizado por Sophia
- Requiere la instalación del paquete elasticsearch

In [1]:
!python -V

Python 3.8.10


In [2]:
!pip install elasticsearch

In [3]:
from elasticsearch import Elasticsearch
from datetime import datetime

## 1. Parámetros de la búsqueda

Características del motor de búsqueda:  
- El formato de las fechas es: YYYY-MM-DD
- El motor de búsqueda incluye noticias por distintos paises, pero se recomienda utilizar "chile" por el momento.
- El motor de búsqueda incluye datos entre el año 2000 y el año 2021.

In [4]:
country="chile"
from_="2010-01-01"
to_="2021-12-31"

keyword="biodiversidad"
simple_keyword=True

## 2. Parámetros de conexión al motor de búsqueda de Sophia

In [5]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "dummy_elastic"

In [6]:
#NO MODIFICAR

es = Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

## 3. Búsqueda de noticias de prensa

- Preparación de la consulta en el formato solicitado por Elasticsearch

In [7]:
#NO MODIFICAR
match=""
if (simple_keyword):
    match="match"
else:
    match="match_phrase"

query = { 
    "bool": { 
      "must": [
        {match: { "text":keyword}}

      ],
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }} 
    ]
    }  
}

- Envio de la consulta
- No se puede devolver más de 10.000 noticias. En caso de obtener 10.000 respuestas, se recomienda buscar en un periodo de tiempo más corto o modificar la consulta.

In [8]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 4283 noticias encontradas...


## 4. Guardar los datos en un dataframe Pandas y en un archivo CSV

In [9]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[],'search':[]}

df = pd.DataFrame(data)  
  
for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    search = keyword
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date, 'search':search}
    
    df = df.append(new_row, ignore_index=True)

df

,id_news,country,media_outlet,url,title,text,date,search
0,22000913.0,chile,elciudadano,https://www.elciudadano.com/actualidad/disponi...,Disponible online importante catálogo con semi...,"La Fundación BIODIVERSIDAD ALIMENTARIA, señala...",2021-09-16,biodiversidad
1,6414845.0,chile,latercera,https://www.latercera.com/que-pasa/noticia/no-...,"""No es suficiente"": Informe reconoce que Chile...","""Si el 2019 fue el año del cambio climático, s...",2020-01-20,biodiversidad
2,15637554.0,chile,emol,https://www.emol.com/noticias/Tecnologia/2016/...,Caída de la biodiversidad en el planeta supera...,MADRID.- La biodiversidad del planeta ha caído...,2016-07-14,biodiversidad
3,5157141.0,chile,elciudadano,https://www.elciudadano.com/chile/congreso-ava...,Congreso: Avanza votación de proyecto que crea...,La Comisión de Medio Ambiente de la Cámara de ...,2020-11-12,biodiversidad
4,21494220.0,chile,elrepuertero,https://www.elrepuertero.cl/noticia/sociedad/e...,En Santo Tomás abordarán la temática del manej...,Porque cada vez se hace más necesario analizar...,2017-11-20,biodiversidad
...,...,...,...,...,...,...,...,...
4278,4946124.0,chile,elciudadano,https://www.elciudadano.com/politica/debate-pr...,"Debate Presidencial en Chile, ¿Qué intereses r...","El Ciudadano entrega aquí las voces, los argu...",2013-06-24,biodiversidad
4279,5006851.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/la-...,La renovación de la crítica al desarrollo y el...,De a poco el Buen Vivir ha ido permeando las ...,2012-03-27,biodiversidad
4280,5703004.0,chile,elciudadano,https://www.elciudadano.com/organizacion-socia...,Poderes populares en América Latina: pistas es...,Compartimos la Introducción del libro colect...,2013-04-13,biodiversidad
4281,5654115.0,chile,ahoranoticiasmega,https://www.meganoticias.cl/nacional/226074-cu...,Cuenta Pública: Discurso completo del Presiden...,MENSAJE PRESIDENCIAL CUENTA PÚBLICA 01/JUNIO/2...,2018-06-01,biodiversidad


- Verifiquemos si existen noticias con la misma URL

In [10]:
df['url'].duplicated().any()

True

- Eliminar las noticias duplicadas

In [11]:
df = df.drop_duplicates(subset='url', keep='first')

- Guardar los archivos en un CSV

In [12]:
nombre_archivo=keyword.replace(' ', '')+"_"+country+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(nombre_archivo)
df.to_csv("./"+nombre_archivo)

El archivo se llama:
biodiversidad_chile_2010-01-01_2021-12-31.csv


- Realizar algunas consultas SQL sobre el dataset

In [14]:
from pandasql import sqldf 

In [16]:
result = sqldf("SELECT media_outlet,count(*) FROM df GROUP BY media_outlet ORDER BY count(*) DESC")
result

,media_outlet,count(*)
0,elciudadano,1375
1,latercera,421
2,laestrelladechiloe,329
3,emol,315
4,biobiochile,275
5,elllanquihue,193
6,elheraldoaustral,176
7,elrepuertero,156
8,radiopudeto,127
9,elhuemul,119
